# Prelude: Using Jupyter Lab

If you know all about Jupyter Lab already, skip to the [PM sensor data](#PM-Sensor-Data) section. If you have run this notebook before or an experienced Pandas user you might even want to jump [straight to the main action](#From-zero-to-plot).

## The most important things to know
... but you don't need to know it all at once (or at all). Read the first bit and skim over the rest if you feel like it. In case you get lost  or confused you can always come back to start of the notebook.

If you are already familiar with Jupyter Lab you might want to skip to the [start of the actual example](#PM-Sensor-Data).

### Running code
Jupyter notebooks are based around the concept of a *cell*. A cell can be one or more lines of code or a long text like this one. When you click on something in this notebook, you'll see a blue bar on the left side, that highlights the current cell.

To execute a cell hit `Shift + Enter`. This moves the focus to the cell below, making it easy to execute a notebook from top to bottom. There are more options, which you can find in the `Run` menu entry above, but this gets you really far.

If the text starts looking weird somewhere, you probably opened a text cell and are in the Markdown edit mode now. Executing that text cell (the same way you execute a code cell) makes it look nice again.

In [ ]:
# click on me and run me with Shift + Enter
print('execute me!')

### Tab completion and documentation access
The other most useful things to know are:
- `Tab` for completing what you are typing in a code cell
- `Shift + Tab` for getting a description of the function or object (with the cursor inside or on the parenthesis for functions), or:
- `Ctrl + i` (or by choosing "Help" -> "Show Contextual Help" from the top menu) you get a new window that will display the help text for whatever you click on. If you click on a variable it also show you its type and the contents (in abbreviated for if necessary). Drag that window to the side or bottom by clicking and holding tab name of the window, so that you can see the notebook at the same time.
- `Ctrl + Shift + c` (or "View" -> "Activate Command Palette" in the top menu) lets you search for anything else and show the keyboard shortcuts for it

In [ ]:
# position the curser after "up" and hit tab
'complete me'.up
# and don't forget to add () to the completion to actually execute the function

In [ ]:
# move the cursor to the end of the next line and hit tab
'complete me'.
# then select one of the functions and call it, e.g. title()

In [ ]:
# move the cursor inside ( ) and hit Shift + Tab
round()
# or look at me with the Contextual Help window (Shift + i)

Small tips and tricks:
* If you hit `Tab` right after a `.` behind a Python object and wait a bit, you can also search for completions that don't start with what you type, e.g. in the example above if you hit tab after typing `'hello'.` and type `up`, you'll get both `upper` and `isupper`. If you hit tab after `'hello'.up` it only completes it to `upper` for you.
* The tab completion and also the help text does not always show up as expected. This most often occurs when you chain several function calls, but also in other situations. In this case a workaround that nearly always works is to assign the intermediate result to a (throwaway) variable and then get the completion or help by using this new variable.

### Moving around quickly

You can open the table of contents for the notebook by clicking on the symbol with three lines (with small dots on the side) symbol in the left toolbar. You can move to a section by clicking on its title there. This section list is generated automatically from the Markdown headings, with the indentation (and optional numbering) based on the how many `#` are used for the heading.

If you right click on the tab of the notebook, you can also select "New View for Notebook", which opens a second window for the same notebook. You can display them side by side and be at different positions in each of them and even run code or change text in either. This can be really handy for example two similar looking code cells that produce slightly different output or for comparing plots. By the way, you can also create a new window for a plot (or any other output, like a dataframe table), by right clicking on it as well.

### Python session and restarting things
The last thing that might be important to know is what the number in the `[ ]` left of each code cell means. It shows how many code cells have been executed when that cell was executed most recently. If it is empty, you haven't executed it yet. This can be useful to figure out if you already executed a cell after updating another one.

It is possible to restart your Python session. This session is called a `kernel` in Jupyter for rather technical reasons. You can restart the session under "Kernel" -> "Restart Kernel" (or click on the nearly closed circle symbol in the toolbar).

Beware though: the code cell execution counts described above stick around from the last session until you execute the cell again in the new session, unless you "Clear All Outputs" as well when restarting the kernel or afterwards (but that also removes your plots and other outputs in the notebook from the previous sessions that might still be useful).

Sometimes you also want to stop Python from doing something, e.g. because you made a mistake and what it now tries to do takes way too long. For that go to "Kernel" -> "Interrupt Kernel" (or click on the filled black square symbol in the toolbar).

# PM Sensor Data
Finally we get to the actual example of analyzing the [PM (Particulate Matter)](https://en.wikipedia.org/wiki/Particulates) measurements from a low cost sensor placed in Lustenau, Austria.

After going through this example you might want to try going this notebook again with a [sensor closer to you](https://maps.sensor.community/#10/47.3426/9.8812) and adjusting the code as needed. For background information on the sensors or the whole citizen science project you can head over to [sensor.community](https://sensor.community/en/).

### The fast lane
If you are not interested in the data import and validation details (yet) or are already an experienced Pandas user and just want to see the code without explanation, you can skip straight to the [visualizing and inspecting the data](#From-zero-to-plot) section.

## Standard imports and configuration

A few lines of boilerplate code you see at the beginning of nearly every Python file for data analysis or machine learning. You can just copy it for now, all you need to know is that the packages will be available under the abbreviated name if you do it like this.

At least the commonly used python package names are usually abbreviated to avoid always having to type the name over and over again. For pandas, numpy and pyplot basically everyone uses the same abbreviations, so you might as well get used to them. For nearly all matplotlib operations the `pyplot` component is used, so I wouldn't bother to abbreviate the main package name.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

The next cell is for configuring matplotlib to work well in the notebook. The first line you might need (if matplotlib uses a different plotting backend for some reason), but the others you can leave away.

I often don't bother with them and the `retina` figure format I discovered just recently (but it does make the plots look much nicer). The default figure size is too small for my taste, so if I leave that line away I keep repeating `figsize=(x,y)` all over the place. If you copy some other boilerplate anyways, you can just include that as well. If you keep using the same values, you can also set it as new default values for all notebooks in a configuration file.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats=['retina'] # for high resoultion images
plt.rcParams['figure.figsize'] = (12,6) # default plot size - play with it

## Loading all sensor data
### Getting the data
#### The fast and easy way (pre-prepared)
Just use the data I already prepared for you. It is just all the CSV file for this sensor (one per day) combined into one CSV file. I then zipped it to make it a bit smaller.

In [ ]:
sensor_file_path = 'data/sds011_4365_combined.csv.zip'

#### The slow and tedious way

You can safely [skip this section](#Reading-the-data) or even skip to the [plotting action](#From-zero-to-plot) if you are not interested in how you might want to do this in a similar situation. Below is a high-level description of how one get all the data and the nitty-gritty details and a walkthrough is available as a separate notebook.

##### **Generating the download URLs**

The URLs for the CSV files all follow the same pattern. Apart from the sensor type name and the sensor id we also need to know the dates at which the sensor reported data. This list of date is not available (as far as I know), so we just need to try all dates from the first date in the archive.

This list of dates can be generated easily with pandas using:
```python
pd.date_range(first_date, last_date, freq='D')
```

##### **Downloading data from the generated URLs**
There are many different ways to download the data. Choose whichever is most comfortable and suitable in your situation. We can:
* use pandas directly, just passing the URL to the `pd.read_csv` function.
* write all the URLs to a text file and then use another program (that you can call directly from Jupyter with `!program_name`) to download them all like `wget` on Linux
* loop over all the URLs, then download the file and save the contents to a local file. The downloading and saving can be done with an external tool (like `curl`) or directly from Python.

##### **Combining the data**
* If we read the date into pandas dataframes we have to concatenate the dataframes into a single dataframe. For this we can use the `pd.concat` function.
* If we downloaded all the original CSV files, we can either loop over all of them and:
  - read them into Pandas dataframes and then concatenate them (see above)
  - read them as text files, remove the header and write the rest of the file into one output csv file

### Reading the data
#### Step-by-step
Reading all the data in and then checking if our assumptions are valid. You can also skip to doing it [all at once](#loading-data-all-at-once).

In [ ]:
sensor_file_path

In [ ]:
data_all = pd.read_csv(sensor_file_path, sep=';')
data_all

##### Checking if columns are empty and dropping useless data
We can already apply what we about this csv file from what we figured out in the single day analysis. But let's quickly confirm our assumptions first. With `count` we can see how many non-empty values the columns contain.

In [ ]:
data_all.count()

As expected, we can get rid of the columns without any values (`durP1`, `ratioP1`, `durP2`, `ratioP2`) as they don't provide any information.

In [ ]:
data_without_empty = data_all.dropna(axis='columns', how='all')
data_without_empty.head(3)

##### Checking how many unique values are in columns
Let's check how many unique values are in the columns that are left. We know already that our `selected_columns` contain a lot of interesting values, so let's temporarily exclude them for this with the `drop` method.

In [ ]:
selected_columns = ['timestamp', 'P1', 'P2']
data_leftover = data_without_empty.drop(selected_columns, axis='columns')
data_leftover.head(3)

In [ ]:
data_leftover.nunique()

##### Counting how often each value appears
We could also look at how many value combinations for the remaining columns exist and how often these combinations appear. If we call `value_counts` on a data frame it gives you all unique rows and how often they appear (unless you restrict it to subset of the columns with the first parameter).

In [ ]:
all_combinations = data_leftover.value_counts()
all_combinations

More commonly `value_counts` is used on a single column (or on a *Series* in Pandas terms). This is especially useful with string columns, e.g. columns containing country names or in this case sensor type names.

In [ ]:
data_leftover['sensor_type'].value_counts()

In [ ]:
data_leftover['lon'].value_counts()

The coordinate longitude data changed slightly over time for some reason. Due to privacy considerations the coordinates in the sensor.community data are not exact anyways (rounded to a resolution of about 100m). Maybe they changed their rounding algorithm a bit at some point.

<a id="loading-data-all-at-once"></a>
#### All at once
We can save memory and speed up the reading a bit by telling Pandas to use only the columns we are interested in with the `usecols` parameter.

We can parse the datetime columns right away with `parse_dates`. Be careful to pass it a *list* of column names as it doesn't accept just a single name as a string (unlike many other Pandas methods). According to the help text it should automatically try to parse all datetime columns if you pass it `True`, but in my limited experience this didn't work.

There is a **lot** more parameters available for `read_csv`. I recommend to have a short look at it (with `Shift + Tab` or better yet the contextual help with `Ctrl + i` if you haven't opened it already), just to get a bit of an idea how powerful it is. For German speakers `decimal` and maybe `thousands` could be needed rather often.

In [ ]:
selected_columns = ['timestamp', 'P1', 'P2']
data_selected = pd.read_csv(sensor_file_path, sep=';',
                            usecols=selected_columns,
                            parse_dates=['timestamp'])
data_selected

### Data preparation
Let's first shorten the variable name as we'll use it a lot. If we want, we can `copy` the original dataframe. This can be useful if we might modify the data frame under this new name, but want to keep the original data around, e.g. to get back to it (without doing everything all over again).

We also make the datetime column become our new row index. Datetime indices are very handy, as this gives us easier access to time ranges, make selecting dates faster and we can leave away the column name for the x-axis when we use Pandas plotting functions for time series plots.

In [ ]:
data = data_selected.copy() # copy is optional
data = data.set_index('timestamp')
print(data.columns)
data.columns = ['PM10', 'PM2.5']
data

In [ ]:
data.plot()

## Validating the data
### Datetime index

Having a sorted index speeds up some operations and prevents misleading results if we select date ranges. Our index looks sorted, but let's check to be on the save side.

The name for checking if an index (or column) is sorted is not very intuitive. Instead of something like *is_sorted* it is called `is_monotonic`. In case you are not familiar with the term, it is mostly used in University level math (or at least this is where I first encountered as a non-native speaker).

In [ ]:
data.index.is_monotonic

#### Detour: checking why the datetime is not sorted
You can safely skip to [Sorting the datetime index](#Sorting-the-datetime-index) if you are not interested.

A bit of a surprise, it is not sorted, although it very much looks sorted.

We can just sort the index and be done with it. You can skip the exploration below and do just that, but often looking into why one assumption is false leads to discovering other false assumptions or peculiarities in the data.

Still with me? Then let's dig a bit into where and why this assumption fails. We start of by looking at the time span between two consecutive entries in the timespan data.

In [ ]:
time_diff = data.index.to_series().diff(1)
time_diff

In [ ]:
time_diff.describe()

Okay, the very minimum and maximum are quite different than the rest. Let's see if this applies only to the most extrem values or if it is not that uncommon.

In [ ]:
time_diff.describe(percentiles=[0.001, .01, .1, .25, .5, .75, .9, .99, .999])

The vast majority of the data was recorded roughly every 2 1/2 minutes. The larger gaps are rather rare.

##### Interlude: boolean vectors and counting True entries
To see how many values are actually negative, we use a bit of a trick. The boolean vectors we get as a result of comparisons can also be treated as integers with 1 for `True` and 0 for `False`. Hence, to get the count of the `True` entries, we can just sum all the numbers together.

In [ ]:
time_diff < pd.Timedelta(0)

In [ ]:
(time_diff < pd.Timedelta(0)).sum()

So, just one position where an earlier timestamp follows a later timestamp.

To see which entry that actually is, we can use the boolean vector in our dataframe selection.

In [ ]:
data[time_diff < pd.Timedelta(0)]

In [ ]:
data.loc['2018-04-02']

Let's get the row numbers by switching back to the standard row index.

In [ ]:
data.reset_index().head()

##### Interlude: selecting data with df.query()

Selecting data with boolean conditions like this `df[df['column'] == 10]` can get annoying quickly, especially if you have multiple conditions and it is not possible to do it this way if you are working with a temporary dataframe that you haven't assigned to a variable yet.

Using multiple conditions with boolean vectors can actually easily trip one up. Due to the order in which the operators like `==` and `&` are evaluated, we have to put parenthesis around the conditions to connect them with `&` (and) or  `|` (or). E.g.
```python
df[(df['column'] > 10) & (df['column'] < 20)]
```

If we forget the parenthesis, depending on the exact conditions we'll get an error or even worse, we don't get an error, but also not the result we intended!

An alternative way to select data is with `df.query('column == 10')`. You pass the function a string in which you can use the column names directly and also use number or string literals. If you include strings though you need to quote them still, but with the two different quote styles in Python it is not that hard: `df.query('sensor_type == "sds011"')`

Multiple conditions are easy with this syntax as well:
```python
df.query('10 <= column_name < 20')
df.query('a == "passed" and status_code > 10')
```

One limitation though of the query syntax is that the column name must be a valid python variable name. For example, we couldn't query our `PM2.5` column with it as dots are not allowed in variable names (and neither are spaces).

In [ ]:
data.query('PM10 > 800')

In [ ]:
data.reset_index().query("timestamp == '2018-04-02 00:01:02'")

With `df.iloc` instead of `df.loc` we can access the rows by their row number (0-based) instead of their labels. If your index also uses integers but doesn't match the row numbers, you have to be careful with which one you use.

In [ ]:
data.iloc[148451:148457]

We suddenly jump from 2 April midnight to the start of the same day. Let's see what happens at the end of the previous day.

In [ ]:
data.reset_index().query('timestamp > "2018-04-01 23:55"').head(5)

Hmm, it jumps from 1st of April midnight to just before noon on the 2nd. We are getting closer to the mystery.

##### Interlude: [making Heads or Tails of](https://idioms.thefreedictionary.com/make+heads+or+tails+of) our timestamp mystery
To look at what happens at the start of the next day, 3rd of April, and a bit before that (where we don't know what to expect) we can do the following:

We select all data up to the end time we know (3rd April, after midnight) and then look at the last few entries before it with `df.tail(n)`, which shows `n` rows from the end, i.e. the opposite of what `head` is doing.

In [ ]:
data.reset_index().query('timestamp < "2018-04-03 00:10"').tail(8)

Looks like we have the following situation:
* 1st April (normally)
* 2nd April 11:50ish - midnight
* 2nd April midnight to before 11:50
* 3rd April (normally)

When we look at the [original CSV file for that day](https://archive.sensor.community/2018-04-02/2018-04-02_sds011_sensor_4365.csv) we see that the timestamps are already messed up there. If we look at other sensors, we see the same pattern on that day, so it very much looks like the archive server was mixing up the row order on that day for some reason.

#### Sorting the datetime index
Luckily for us the solution is much shorter than the investigation was:

In [ ]:
%%time
data = data.sort_index()
data.index.is_monotonic

### Time zone handling

The sensor data does not include any time zone information and in the archive description it is not mentioned either.

So, what time zone do we have? Is it the local time? If so, does it switch between summer and winter time?

In some forum comments it is mentioned that the datetime is using the [UTC, the Coordinated Universal Time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), so the offset to the local time in Austria is 1 hour in the winter and 2 hours in the summer.

Working with time zones can be really painful, especially if one needs to coordinate between different formats. Luckily, Pandas is aware of time zones and does most of the work for us.

#### Adding time zone information

If we have datetimes without any time zone information, we first need to tell Pandas which time zone was used with `tz_localize`. Once this is done, we can convert it to whatever time zone we want with `tz_convert` (and if you forget the first step it won't let you do the second one).

In [ ]:
data.tz_localize('UTC') # adds +00:00 to the timestamps

In [ ]:
data = data.tz_localize('UTC').tz_convert('Europe/Vienna')
data

Switching between summer and winter time happens at the end of October. Let's see if Pandas handled it properly:

In [ ]:
data.loc['2020-10-24':'2020-11-01']

Looks good, +02:00 in summer, +01:00 in winter.

What time zone should be used for the analysis is actually not that clear cut. By looking at the local time we will likely preserve more the social aspects in the data (e.g. people commuting at the same clock time) and if we don't use the summer time change we should preserve more the natural aspects (e.g. sunrise time and how that might affect wind patterns).

It could actually be interesting to look for PM pattern changes around the switching dates between winter and summer time. Let's stick with the local time for now, as we probably don't have enough data so far for a meaningful analysis.

### All at once

Let's do all the little changes we made above in one step. Otherwise it might happen, that we forget one or mix up the order at some point. It's also a nice review and closer to what an experienced Pandas user would do.

In [ ]:
data = data_selected.copy() # make a copy of the original data (optional)
data = data.set_index('timestamp') # to access the data by datetime
data.columns = ['PM10', 'PM2.5'] # rename columns from "P1", "P2"
data = data.sort_index() # fix a small problem in the order of the datetime info
data = data.tz_localize('UTC').tz_convert('Europe/Vienna') # use local time
data

## Visualizing and inspecting the data
### From zero to plot
In case you jumped to this section or messed up your dataframes, below is the code for loading all data and get it into the shape for plotting. If you executed all the relevant stuff so far, you don't need to execute this cell.

The code is hidden by default as we have seen all before in detail. If you want to see it anyways, click on the three dots or "View" -> "Expand Selected Code" from the menu.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats=['retina'] # for high resoultion images
plt.rcParams['figure.figsize'] = (12,6) # default plot size - play with it

sensor_file_path = 'data/sds011_4365_combined.csv.zip'
selected_columns = ['timestamp', 'P1', 'P2']
data_selected = pd.read_csv(sensor_file_path, sep=';',
    usecols=selected_columns, parse_dates=['timestamp'])

data = data_selected.copy().set_index('timestamp').sort_index()
data = data.tz_localize('UTC').tz_convert('Europe/Vienna')
data.columns = ['PM10', 'PM2.5']
data.head(3)

### Exploring the values over time
Now we are finally all prepared to look at the data. I usually start off by plotting it (or calculating summary statistics with `describe`).

In [ ]:
data.plot() ;

That's a good overview of what data we have available. We can see range and can already spot some gaps in the data, but we can't see any details. Let's zoom in a bit.

#### Selecting time ranges
Selecting a date or time range is a breeze in Pandas with datetime indices:

In [ ]:
data.loc['2020'].plot() ;

In [ ]:
data.loc['2021-05'].plot() ;

You might have noticed that the end date was included fully. Unlike other slices in Python and Pandas, datetime slices include the end of the range as well.

In [ ]:
data.loc['2021-05':'2021-05-07'].plot() ;

In [ ]:
data.loc['2021-04-30':'2021-05-02'].plot() ;

Let's zoom out a bit again and cut off the highest spikes, so we see more of the actual pattern and make the curves a bit transparent so that we can see the $PM_{10}$ values hidden behind the $PM_{2.5}$ values as well.

In [ ]:
data.loc['2020'].plot(ylim=(0, 200), alpha=0.8) ;

#### Interlude: Interactively selecting the date ranges to plot

It is a lot of data to look at. Instead of keeping changing the date selection string in the code, we can do this more easily with widgets.

To do that, we need to create the plotting function for it first, but if you want to just see the interactive widget in action, you can [jump to it](#interactive-timeplot). We want to be able to plot an offset with a fixed number of days. With the strings we have been using so far, this is a bit difficult (we would need to consider how many days are in a month,...).

Let's start with creating our starting date first from variables. Python has a couple of ways for formatting strings, but the easiest is putting an `f` before the opening quote and then writing the variable names (or Python expressions) inside curly braces within the string.

In [ ]:
year = 2021
month = 2
day = 17
f'{year}-{month}-{day}'

This string we can pass to Python for creating the datetime object for us.

*Side note: in any kind of production or polished code you wouldn't want to do this by using strings, but that is what we know so far. You would rather do something like this:*
```python
from datetime import datetime # built-in Python module
start_time = datetime(year, month, day, tzinfo=data.index.tz)
```

In [ ]:
pd.to_datetime(f'{year}-{month}-{day}')

The end time we can get with Pandas offsets. This can actually do much more complicated offset than days (e.g. business month begin or Easter).

In [ ]:
pd.offsets.Day() * 14

In [ ]:
start_time = pd.to_datetime(f'{year}-{month}-{day}')
end_time = start_time + pd.offsets.Day() * 14
end_time

When we try to select data with it, we get a slightly scary looking, but actually rather informative warning:

In [ ]:
data.loc[start_time:end_time]

Seems like Pandas is stricter when the datetime range is selected with datetime objects instead of strings. This does make some sense, as the datetime object are more likely to be created programmatically or come from another data source that might use a different time zone, whereas the string form is mostly used interactively.

We already know how to do that, so let's try it. We don't have to convert between time zones, as we can just choose which time zone our date selection should have:

In [ ]:
start_time.tz_localize('Europe/Vienna')

In [ ]:
start_time = pd.to_datetime(f'{year}-{month}-{day}')
start_time = start_time.tz_localize('Europe/Vienna')
end_time = start_time + pd.offsets.Day() * 14
end_time

In [ ]:
data.loc[start_time:end_time].plot(alpha=0.8)

Now we just put that code into a function. All we really need to do for that is give it a name with `def` and indent our code lines by hitting the tab key, like this:
```python
def function_name(parameter1, parameter2):
    do_something_with(paramter1, parameter2)
```

It is possible to provide default values for the parameters. This way we can leave away some or all of them.

In [ ]:
def plot_pm(year=2021, month=5, day=1, days_to_show=14):
    start_time = pd.to_datetime(f'{year}-{month}-{day}').tz_localize('Europe/Vienna')
    df = data[['PM10', 'PM2.5']] # in case we add other columns later
    df.loc[start_time:start_time+pd.offsets.Day()*days_to_show].plot(ylim=0)
    plt.show() # to make sure interactive output updates properly

plot_pm()
plot_pm(2020)
plot_pm(month=6)

##### Interactive plot of PM over time
With the `plot_pm` function we just created above we are now all set for creating our interactive widget.

The ipywidgets module should come with Jupyter. If not remove the `#` to uncomment the second cell below and execute it. This will download and install the package into your Python environment.

In [ ]:
from ipywidgets import interact

In [ ]:
#%pip install ipywidgets

To add interactivity we just tell `interact` which function to use and here it makes sense to also tell it the value ranges we want to accept.

In [ ]:
interact(plot_pm, year=(2017, 2021), month=(1, 12), day=(0, 31),
         days_to_show=(1, 60)) ;

There are a lot of short spikes in the data and generally it looks quite noisy. The time resolution of the data is also higher than needed, if we want to look at many days at once. To make easier to extract information from it we can add some (optional) smoothing to our interactive pot

First, we smooth the data a bit by first creating fixed size time bins. Additionally, we create a version that calculates a rolling average on top of that. The [`resample`](#resampling) and [`rolling`](#rolling) functions will be explained and shown in more detail in later sections.

To select which version of the data we want to plot, we use make use of a dictionary. A dictionary is similar to a list, but instead of just being able to access the value by the position, we can access it by some kind of key. The keys are often strings but can be nearly anything. Dictionaries are used a lot in Python, even for very simple things, like our use of it here. We could write the same thing easily with `if` and else if (called `elif` in Python), but it is a bit shorter this way (especially if we end up adding more options).

They even have a special syntax for creating them. Instead of the `dict()` used below, we could also write it as:
```python
data_versions = {'raw': data, 'smooth1': data_fixed_frequency, 'smooth2': data_rolling_average}
```

In [ ]:
# uncomment the following line if scipy is missing
#%pip install scipy

In [ ]:
data_fixed_frequency = data[['PM10', 'PM2.5']].resample('10min').mean()
data_rolling_average = data_fixed_frequency.rolling(13, center=True,
                                                    win_type='gaussian').mean(std=3)

data_versions = dict(raw=data, smooth1=data_fixed_frequency, smooth2=data_rolling_average)
print(f'Available options: {data_versions.keys()}')

def plot_pm_smoothed(year=2021, month=5, day=1, days_to_show=14, version='smooth2'):
    data_to_use = data_versions[version]
    data_to_use = data_to_use[['PM10', 'PM2.5']] # in case we add columns later
    start_time = pd.to_datetime(f'{year}-{month}-{day}').tz_localize('Europe/Vienna')
    data_to_use.loc[start_time:start_time+pd.offsets.Day()*days_to_show].plot(ylim=0)
    plt.title(version)
    plt.show() # to make sure interactive output updates properly

plot_pm_smoothed(month=6, days_to_show=4)
plot_pm_smoothed(month=6, days_to_show=4, version='smooth1')
plot_pm_smoothed(month=6, days_to_show=4, version='raw')

Quite a difference between the different versions of the data. You might also have noticed that Pandas formats the $y$ axis differently in the smoothed versions. It does that as soon as your datetime index has a fixed known frequency (either created by binning / downsampling with `resample` or if you have by just letting Pandas know about it and fill in any missing time slot with null values with `asfreq`.

Let's add the version selection to our interactive widget, so we can flip between them on the spot, e.g. to check if the raw data is suspicious. For that we just need to tell interact what values are possible (with a list or something similar like the dictionary keys we use here).

In [ ]:
interact(plot_pm_smoothed, year=(2017, 2021), month=(1, 12), day=(0, 31),
         days_to_show=(1, 60), version=data_versions.keys()) ;

#### Interlude: Scatter plot
If we look at the data over a long time period, there are just so many data points and short-term variations in it that a line plot is not the ideal way to visualize it. One alternative to the smoothing we did above is to plot each data point with a small dot in a scatter plot, especially if you are not familiar with the data (quality) yet. Smoothing can make some things, like for example erroneous data, hard to spot as we are not looking at the raw data anymore.

Pandas *scatter plots* require an explicit column name for the x-axis (and it needs to be a column, not an index) and they also don't allow you to plot more than one column on the y-axis:

In [ ]:
data.loc['2020'].reset_index().plot.scatter('timestamp', 'PM10')
data.loc['2020'].reset_index().plot.scatter('timestamp', 'PM2.5')

With this many data points the dots overlap each other strongly as they are too big. We can improve this situation by using a different marker, `.` instead of the default  `o` (small letter O) and making the dots slightly transparent with the `alpha` parameter.

In [ ]:
data.loc['2020'].reset_index().plot.scatter(
    'timestamp', 'PM10', ylim=(0, 200), alpha=0.3, marker='.')
data.loc['2020'].reset_index().plot.scatter(
    'timestamp', 'PM2.5', ylim=(0, 200), alpha=0.3, marker='.')

We can plot both variables in the same plot by telling the second plot to use the "axes" (the drawing area) from the first plot.

In [ ]:
ax = data.loc['2020'].reset_index().plot.scatter('timestamp', 'PM10',
    ylim=(0, 200), alpha=0.1, marker='.')
data.loc['2020'].reset_index().plot.scatter(
    'timestamp', 'PM2.5', ylim=(0, 200), alpha=0.1, marker='.',
    ax=ax, color='C1')

#### Tip: Quick scatter plots with Pandas
Here is a small trick: we can achieve the same thing more easily with the normal plot 
function. We just need to make the lines invisible with `linewidth=0` (or shorter with `lw=0`)

In [ ]:
data.loc['2020'].plot(ylim=(0, 200), alpha=0.1, marker='.', linewidth=0)

We can also plot them separatly if we feel like it:

In [ ]:
data.loc['2020'].plot(ylim=(0, 100), alpha=0.1, marker='.', lw=0, subplots=True) ;

If you want to get closer to the two scatter plots above, we can give them both the same color and make the plots a bit taller:

In [ ]:
data.loc['2020'].plot(ylim=(0, 100), alpha=0.1, marker='.', lw=0, subplots=True,
                      color='C0', figsize=(12,12)) ;

### Exploring the value distribution
Now that we have looked a bit how the measurements change with time, it is time (no pun intended) to look at measurements themself in more detail.

We can get useful summary statistics with `describe()` or plot the value distribution in histograms.

In [ ]:
data_stats = data.describe()
data_stats

In [ ]:
data.hist() ;

With our value distribution the default histogram parameters are really not useful. Let's do something abou that:

In [ ]:
data.hist(log=True) ;

In [ ]:
data.hist(log=True, bins=100) ;

#### Relationship between PM2.5 and PM10 values
We have more than one measurement point per time and they measure similar things, so let's have a look at how they relate to each other.

In [ ]:
data.plot.scatter('PM2.5', 'PM10') ;

In [ ]:
data[data['PM10'] < 250].plot.scatter('PM2.5', 'PM10', marker='.', alpha=0.3) ;

The bottom of the cloud is cut off quite sharply. Looking at it more closely it is around where $PM_{2.5}$ and $PM_{10}$ have roughly same value.

Hypothesis: this cut-off is not a measurement artifact. Instead it is due to $PM_{10}$ (particulates $< 10 \mu g$) containing by definition all $PM_{2.5}$ particulates ($< 2.5\mu g$). 

Let's add a line with $x == y$ to the plot. To do this we just need to define the start and end point, e.g. $0$ and $125$ and use the same start and end point for the $x$ and $y$ axis.

In [ ]:
data[data['PM10'] < 250].plot.scatter('PM2.5', 'PM10', marker='.', alpha=0.3)
plt.plot([0, 125], [0, 125], color='orange') ;

Looks promising, but we should zoom in a bit more on the main cloud.

In [ ]:
xlimits = 0, 40
data[data['PM2.5'] < xlimits[1]].plot.scatter('PM2.5', 'PM10', marker='.', alpha=0.3)
plt.plot(xlimits, xlimits, color='orange') ;

Instead of selecting which values we want to show we can use all entries from the dataframe and just set the plot boundaries to the area we are interested in with xlim and ylim.

This works well if you set the limits for both the $x$ and $y$ axis explicitly. However, if you just to limit one axis to a known range, the auto-scaling of the other axis will be based on all of the data, not just the ones corresponding to the limits you set on the other axis. This can result in the other axis having a too large value range. If that bothers either set the other limits based on what you see in the plot explicitly as well or go back to selecting just a subset of the data from the dataframe.

In [ ]:
xlimits = 0, 40
ax = data.plot.scatter('PM2.5', 'PM10', xlim=xlimits, ylim=(0, 120),
                       marker='.', alpha=0.3)
ax.plot(xlimits, xlimits, color='orange', alpha=0.3) ;

In [ ]:
xlimits = (0, 4)
ax = data.plot.scatter('PM2.5', 'PM10', xlim=xlimits, ylim=(0, 12),
                       marker='.', alpha=0.3)
ax.plot(xlimits, xlimits, color='orange', alpha=0.3, lw=3) ;

I think we are close enough. The stripes we see are probably due to the limited resolution of the $PM_{2.5}$ sensor or something strange happening with the binning in the sensor (as there *are* a few data points in between the stripes, just way less).

A few $PM_{10}$ data points are smaller than $PM_{2.5}$, likely due to measurement inaccuracy. However, this implies that $PM_{10} \ge PM_{2.5}$ is probably not enforced in a post-processing step, but due to actual measurements.

#### PM10 particles that are larger than PM2.5
Maybe it makes more sense to just look at $PM_{10}$ particulates that are larger than $2.5 \mu g$ when comparing it with $PM_{2.5}$, as the smaller ones are already contained in the latter measure.

In [ ]:
data['PM10-2.5'] = data['PM10'] - data['PM2.5']

In [ ]:
data[data['PM10'] < 1000].plot.scatter('PM2.5', 'PM10-2.5', marker='.', alpha=0.3) ;

In [ ]:
data[data['PM2.5'] < 100].plot.scatter('PM2.5', 'PM10-2.5', marker='.', alpha=0.3,
                                       ylim=(-1, 200)) ;

So far, our plots have been far wider then taller, which lead to a noticeably smaller scale on the $y$ axis. This can lead to misleading visual impressions, so let's try to fix that.

In [ ]:
ax = data[data['PM2.5'] < 100].plot.scatter('PM2.5', 'PM10-2.5', grid=True,
                                            marker='.', alpha=0.3, figsize=(12,12))
ax.set_ylim(-1, 200)
ax.set_yticks(range(0, 201, 10))
ax.set_aspect('equal')

### Aggregating data over longer time periods
#### Differences between years

<a id="groupby"></a>
##### Interlude: Grouping subsets of data with groupby
Often we want to compare different subsets of the data with each other. Pandas makes this rather easy with the `groupby` function. What it does is to select the subsets for you, based on the criteria you give it.

Afterwards you optionally select which columns you want to work on and you select an operation you want to do on each of the subgroups. This is usually an aggregation of some kind, like taking the mean, but you could also do some transformation of the data, like subtracting the group mean from the values in each group separately.

Usually the grouping is done on categorical data in one of the columns, like for example sensor id or sensor type name if we had data from more than one sensor. We don't have anything like this here, however, datetime columns (or indices) allow you to easily pull out a part of the datetime information (like year or hour):

In [ ]:
data.index.year.value_counts()

In [ ]:
grouped_by_year = data.groupby(data.index.year)
grouped_by_year.ngroups

In [ ]:
grouped_by_year.mean()

If the aggregation we choose return more than one column, we suddenly have two levels of columns:

In [ ]:
year_stats = grouped_by_year.describe()
year_stats

If you look at each of the individual original columns separately, it is not that hard to work with it, because as soon as we select one of the outer / upper column values we get back a dataframe like we are used to:

In [ ]:
year_stats['PM2.5']

However, selecting one (or more) column names from the lower level for all of the upper levels (like getting the mean for all of our PM values) is a bit trickier. We will discuss that a bit later [here](#hierarchical-column-index-part1) and [here](#hierarchical-column-index-part2).

In [ ]:
plot_cols = ['min', '25%', '50%', 'mean', '75%']
ax = year_stats['PM2.5'].plot.bar(y=plot_cols) # max is too high!

###### Mini-Interlude: Getting the position of a value in a list: enumerate
We can add the same statistic measures over all years to the plot for a better comparison. For this we use `enumerate`, which takes lists (or other "iterables") and returns a list of tuples `(i, original_value)`, where `i` is the count of values from the original list up to that time (which is also its index in the original list).

To get to see the values we need the turn it into a `list` (or loop over it), as enumerate is evaluated lazily, so just returning the values one by one as you ask for it.

In [ ]:
list(enumerate(plot_cols))

The matplotlib standard colors of the active style are accessible with `C0`, `C1`,... For the default style we have used so far, the first color `C0` is blue and the second `C1` is orange, and so on.

In [ ]:
plot_cols = ['min', '25%', '50%', 'mean', '75%']
ax = year_stats['PM2.5'].plot.bar(y=plot_cols) # max is too high!
for i, stastistic_name in enumerate(plot_cols):
    ax.axhline(data_stats.loc[stastistic_name, 'PM2.5'], color=f'C{i}', alpha=0.5)

Looks good already, but the labels on the $y$ axis are spaced a bit too far apart to estimate the values well. We can tell matplotlib at which $y$ values to place the labels with `ax.set_yticks` (or `plt.yticks`).

###### Mini-Interlude: Generating a list of regularly spaced numbers: range
I'm too lazy to type out the numbers from 1 to 13, but I don't have to. The built-in Python function `range` gives as back a range of integers. If we just give it a number $n$ it gives as the numbers from 0 to $n-1$. This might take a bit getting used to, but the upper end of ranges or slices in Python are not included.

However, this actually works quite well with $0$-based counting in Python: if we call `range(n)` we do get `n` values (just each one lower than you might expect). This also works well if you do need the numbers as an index into a list or so. Although explicit indexing is usually avoided in Python, sometimes you do need it and the typical way to do a C-style for-loop is this:
```python
for i in range(len(a_list_or_so)):
    do_something_with(a_list_or_so[i])
```

If you want to create ranges to use with Pandas, there is a very similar function in `numpy` called `arange`. It works the same way but returns a numpy array (and is not evaluated lazily).

In [ ]:
list(range(5))

In [ ]:
list(range(3, 9))

In [ ]:
list(range(9, 2, -2))

In [ ]:
plot_cols = ['min', '25%', '50%', 'mean', '75%']
ax = year_stats['PM2.5'].plot.bar(y=plot_cols) # max is too high!
for i, stastistic_name in enumerate(plot_cols):
    ax.axhline(data_stats.loc[stastistic_name, 'PM2.5'], color=f'C{i}', alpha=0.5)

upper_ylimit = ax.get_ylim()[1]
ax.set_yticks(range(int(upper_ylimit) + 1)) ;

We can also put some part of the date information into a new column, for example if we want to use a Pandas function that only takes column names:

In [ ]:
data['year'] = data.index.year
data.head(3)

<a id="boxplots"></a>
###### Interlude: Boxplots
Boxplots are a tool often used in statistics to visualize data distributions. Let's try it for our year data:

In [ ]:
data[['PM10', 'PM2.5', 'year']].boxplot(by='year') ;

Doesn't look that informative and we can hardly see the weird stuff that is drawn at the bottom. This is due to our distribution having a long tail of much higher values. If we limit the maximum measurement value it looks much more useful:

In [ ]:
axes = data[['PM10', 'PM2.5', 'year']].boxplot(by='year') ;
axes[0].set_ylim(0, 50)

If you don't know what you are seeing and are interested in it, below is a not so short, but hopefully easy to understand explanation. Otherwise move on to [more boxplots](#per-month) or [another way of plotting summary statistics](#plot-with-table) or if plots are not your thing to [resampling](#resampling).

The blue boxes show us three things:
* the green line in the middle is the median, so half the values are larger than it and have are smaller.
* The end of the box are the 1st and 3rd quartile or 25% and 75% percentile. It is very similar to the median, just that it is not half and half, but 1/4 smaller and 3/4 larger for the 1st quartile and the other way around for the 3rd quartile.

All of these values we looked at a couple of times already when we called `describe`. Here they are just visualized to compare the values more easily. The information we can get from the boxes is for example that the data is not distributed evenly, because the green line (median or 50% value) is clearly not in the middle of the box.

The stuff that is sticking out from the box is a bit trickier to understand. They are used to show how far the data reaches. For example, here you see at the lower end they always end at (or very close to) zero.

However, you often have values in your data that are far outside the typical range. These are often called outliers. To not give them too much weight in the visualization they are not included in the thing sticking out from the box and instead are drawn separately as circles.

At what point they are excluded is actually not always based on the same measure. In Pandas it is if they are more than 1 1/2 times further away from the box then the box is long. Seen another way, it takes more than 150% the space to reach them from where half the values are located, then the space which this half of the values take up.

<a id="per-month"></a>
#### Over months (seasonality)
Variations within a year.

In [ ]:
data['month'] = data.index.month

In [ ]:
ax0, ax1 = data.boxplot(['PM10', 'PM2.5'], by='month')
ax0.set_ylim(0, 700)

In [ ]:
ax0, ax1 = data.boxplot(['PM10', 'PM2.5'], by='month')
ax0.set_ylim(0, 80)
ax0.axhline(data['PM10'].median(), color='orange')
ax1.axhline(data['PM2.5'].median(), color='orange')

In [ ]:
month_stat = data['PM2.5'].groupby(data.index.month).describe()
month_stat.rename_axis(index='month', inplace=True)
month_stat

<a id="plot-with-table"></a>
##### Plotting summary statistics over month
Once more calculating the month statistics if you skipped it:

In [ ]:
data['month'] = data.index.month
month_stat = data['PM2.5'].groupby(data.index.month).describe().\
    rename_axis(index='month')

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(marker='o')

What a confusing mess. Let's move the `max` data to the right $y$ axis and make it use a different scale (or we just give up on the max).

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(marker='o',
                                                    secondary_y='max')

Better, but still hard to read as hell. Maybe now you start to appreciate [boxplots](#boxplots) a bit more?! Although to be fair, we are attempting to visualize a bit more information than in the boxplot (i.e. the mean value and standard deviation).

But let's not give up already. Making the curves easy to distinguish from each other could help a lot. Time for styling and a small Python trick:

In [ ]:
['a', 'b', 'c'] * 3

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(
    secondary_y='max', marker='o', style=['-', '-.', ':']*3,
    colormap='Spectral', figsize=(12, 8))

Better, but still far from great :(

For styling advice I recommend to look at the [matplotlib example gallery](https://matplotlib.org/stable/gallery/index.html) and for a listing of [color schemes this example](https://matplotlib.org/stable/gallery/color/colormap_reference.html#sphx-glr-gallery-color-colormap-reference-py) in particular.

The yellow is hard to see, but most colormaps have some light-colored component or otherwise they are not distinguishable enough. We'll see later how to do [some more easy styling](#plot-styling), like using a darker background. Marching on, maybe we can communicate the information in a different way:

##### Interlude: Adding exact values as tabular data to plot
The plot above is overloaded and confusing. Additionally, we don't get to see the exact values. Let's try to fix that step by step.

First, we can use the handy `table` parameter of the pandas plot function:

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(secondary_y='max', marker='o',
    style=['-', '-.', ':']*3, colormap='Spectral', figsize=(12, 8),
    table=True)

To avoid overlapping the table with the $x$ axis ticks and label, we can just get rid of them. The ticks we move to the top and the label we just leave away.

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(secondary_y='max', marker='o',
    style=['-', '-.', ':']*3, colormap='Spectral', figsize=(12, 8),
    table=True)
ax.xaxis.tick_top()
ax.set_xlabel('') ;

The number formatting in the table makes it hard to read. By default, it shows the full precision of the numbers. We can make our own table and round the numbers to the precision we want to have in the plot.

Also, month names instead of numbers would be useful. Instead of typing all the month names, we can get the information from the built-in `calendar` module. That module can actually do quite a bit more than that:

In [ ]:
import calendar
month_table = np.round(month_stat, 2).drop('count', axis=1).T
month_table.columns = [calendar.month_abbr[int(num)] for num in month_table.columns]
month_table

In [ ]:
print(calendar.calendar(2021))

Instead of nicely formatting the calendar and printing it, we can also obtain the dates per week as a list of lists:

In [ ]:
calendar.monthcalendar(2021, 7)

Now we can add our table with the nicer formatting to the plot:

In [ ]:
ax = month_stat.drop(['count', 'min'], axis=1).plot(secondary_y='max', marker='o',
    style=['-', '-.', ':']*3, colormap='Spectral', figsize=(12, 8),
    table=month_table)
ax.xaxis.tick_top()  # Display x-axis ticks on top.
ax.set_xlabel('') ;

Much better, but the main plot is still visually overloaded. Now that we have the numeric data in the bottom though, we can only display the most important curves.

Additionally, we can just show the range of the `25%` to the `75%` percentiles filling the space between with a slight shade instead of displaying the curve.

In [ ]:
ax = month_stat[['mean', 'std', '50%']].plot(
    secondary_y='max', marker='o', style=['-', '-.']*3,
    figsize=(12, 8), table=month_table, grid=True)
ax.fill_between(month_stat.index, '25%', '75%', data=month_stat,
                color='gray', alpha=0.2)
ax.xaxis.tick_top()  # Display x-axis ticks on top.
ax.set_xlabel('') ;

Okay, now it looks actually useable. We could still add labels to describe what the shaded area is and make the are extend the $x$ limits, but let's not overdo it for now.

### Short-term variations
#### Day by day

<a id="resampling"></a>
##### Interlude: Resampling the data to a fixed time interval

So far, the distance between two consecutive data points varies. This makes our life harder than it needs to be in a couple of ways:
* it is harder to access exact time points, as we don't know where they exactly are (if we guess we are likely a few seconds or more off)
* combining our data with another data set (e.g. other sensor data or weather data) is much easier if both have a fixed time frequency
* Last but not least: Pandas does nicer $x$ label formatting for us

Generally, it is quite easy to achieve, we just call `resample` and tell it what time interval we want and then we need to tell it how to combine all the data points in an interval to a single data point, e.g. averaging the values with `mean`.

In [ ]:
%%time
daily_mean = data.resample('D').mean()
daily_mean

In [ ]:
daily_mean.loc['2021'].plot(y=['PM10', 'PM2.5'], lw=1, marker='.')

##### Detour: Groupby vs. Resample
If the difference and uses cases between the two is clear to you, feel free to jump to the [next section](#highest-days).

For resampling to days we could actually also achieve the same thing with [groupby](#groupby), although there are some differences: it takes a bit longer and it doesn't include a frequency information by default.

In [ ]:
%%time
daily_mean_groupby = data.groupby(data.index.date).mean()
daily_mean_groupby.head()

In [ ]:
daily_mean_groupby.head()

In [ ]:
len(daily_mean), len(daily_mean_groupby)

The daily mean from the `resample` contains also entries for the missing dates with the values set to NaN (not a number). This makes it easier for some operations and is what pandas needs for having an index of fixed frequency.

In [ ]:
len(daily_mean.dropna(axis='rows', how='all'))

We can change the daily means from the `groupby` into the same format and frequency information by calling `asfreq`, which adds padding for the missing days and also adds the frequency information to the index.

In [ ]:
daily_mean_groupby.asfreq('D').equals(
    daily_mean.set_index(daily_mean.index.date))

In general, [groupby](#groupby) is used when we have a not too huge number of groups and there doesn't need to be any kind of logical order from one group to another.

In contrast [resample](#resample) can only be used for timeseries data with a datetime index. There is a clear temporal ordering between the groups. It is likely faster than groupby, maybe because it makes good use of this ordering information.

<a id="highest-days"></a>
##### Days with highest average PM2.5

In [ ]:
daily_mean.nlargest(10, 'PM2.5')

In [ ]:
daily_mean.nlargest(50, 'PM2.5')['year'].value_counts()

One thing you might have noticed is that the New Year's Day and New Year's Eve showed up a couple of times among the highest days. Let's look a bit closer at this, by averaging over shorter time spans.

##### Longer peaks within a day / hourly means resampling
From plotting the data in the beginning, we have seen a lot of peaks and some of them are very high. However, most of the peaks have a very short duration. Longer duration peaks are more interesting, as they have a bigger impact on longer term averages (like daily above) and because it is not from a very transient event (or even from a measurement artifact).

In [ ]:
hourly_mean = data.resample('1h').mean()

In [ ]:
hourly_mean.nlargest(15, 'PM2.5')

The keen explorer might have noticed that the resample labels are always at the start (or end) of the frequency period and not in the middle of it. For days it does make sense and for hours to some degree as well, though if you plot them, the values lag half an hour behind what you might expect.

In a short moment we'll look at [how to center the resample labels](#resample-center).

##### Detour: Looking at PM around New Year's Eve
*also known as* "it's not rocket science"

Nearly all of the highest hourly means are actually around New Year's Eve! Let's have a look at it visually.

In [ ]:
data.loc['2018-12-29':'2019-01-03'].plot(y=['PM10', 'PM2.5'])
data.loc['2019-12-29':'2020-01-03'].plot(y=['PM10', 'PM2.5'])

In [ ]:
data.loc['2018-12-31':'2019-01-01'].plot(y=['PM10', 'PM2.5'])
data.loc['2019-12-31':'2020-01-01'].plot(y=['PM10', 'PM2.5'])

In [ ]:
data.loc['2018-12-31 12:00':'2019-01-01 12:00'].plot(y=['PM10', 'PM2.5'])
data.loc['2019-12-31 12:00':'2020-01-01 12:00'].plot(y=['PM10', 'PM2.5'])

The spikes are highest just after the midnight, as we would expect with all the rockets going up. If we hadn't fixed the time zone information, latest now we would have noticed it!

Let's look even closer, but for that we need to decrease our time interval a bit to say 10min.

<a id="resample-center"></a>
##### Interlude: How to center resample labels

One thing to take notice off about `resample` method is that it puts the label at the start or the end of the interval (by default automatically depending on which frequency, e.g. for days at the beginning, but for quarters at the end of it). While we can choose between beginning and end, putting the label in the middle is currently not an option to choose from! :(

We can work around this a bit by shifting the time labels by half the interval before resampling:

In [ ]:
data.head()

In [ ]:
data.resample('10min').mean().head(2)

In [ ]:
data.iloc[:2].mean()

So for the first interval, 23:20, it takes from 23:20 - 23:29:59, so here the first two rows, calculates the mean, and then puts the label at the start of the interval, 23:20, instead of the middle, 23:25, as would make more sense in this case.

It used to be a bit easier, by using the `loffset` parameter in resample, but that is deprecated and will be removed in the future:

In [ ]:
data.resample('10min', loffset='5min').mean().head(2)

In [ ]:
data_10min = data[['PM10', 'PM2.5']].resample('10min').mean()
data_10min.index = data_10min.index + pd.tseries.frequencies.to_offset('5min')
data_10min.head()

<a id="rolling"></a>
###### Mini-Interlude: Rolling average
We can smooth it a bit by taking a rolling average with `rolling` and then `mean`. Again, we have to be careful about putting the label in the middle of the rolling interval, but here we can at least achieve this easily by adding `center=True`.

Taking rolling averages or other rolling statistics (e.g. `max`) is really easy in Pandas and also quite powerful. For example, we can put different weights on the data points within the rolling window with the `win_type` parameter, like a triangular window or a Gaussian bell curve (and many [more advanced window types](https://docs.scipy.org/doc/scipy/reference/reference/signal.windows.html#module-scipy.signal.windows) from the [SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/index.html) [signal processing module](https://docs.scipy.org/doc/scipy/reference/reference/signal.html#module-scipy.signal)). We'll make use of that a bit later.

In [ ]:
data_10min.rolling(3).mean().head()

In [ ]:
data_10min.rolling(3, center=True).mean().head()

In [ ]:
data_10min.rolling(3, center=True).mean().loc['2018-12-29':'2019-01-03'].plot()
data_10min.rolling(3, center=True).mean().loc['2019-12-29':'2020-01-03'].plot()

<a id="subplots"></a>
##### Interlude: Multiple Plots in one figure
Having more than one plot in a figure is actually quite easy with matplotlib. You just call `plt.subplots` and tell it how many rows and columns you want to have and it gives you back the figure (the thing that contains the plots) and the axes (what maptlotlib calls the area for one plot). The Pandas plotting libraries accept an axes to plot into with the `ax` parameter:

In [ ]:
fig, axes = plt.subplots(nrows=2)
data_10min.rolling(3, center=True).mean().loc[
    '2018-12-29':'2019-01-03'].plot(ax=axes[0])
data_10min.rolling(3, center=True).mean().loc[
    '2019-12-29':'2020-01-03'].plot(ax=axes[1]) ;

With `subplots` you can also specify if the plots have the same $x$ or $y$ axis range with the `sharex` and `sharey` parameters. You can also set the figure size with `figsize`.

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(12, 5))
data_10min.rolling(3, center=True).mean().loc[
    '2018-12-29':'2019-01-03'].plot(ax=axes[0], xlabel='')
data_10min.rolling(3, center=True).mean().loc[
    '2019-12-29':'2020-01-03'].plot(ax=axes[1], xlabel='') ;

If you have multiple rows and columns you have to be a bit careful with how to access them, as in that case subplots gives you back an array of arrays / list of lists, where the outer list contains the rows and the inner lists the columns within a row:

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,0.5))
axes

Often you just want to create one subplot after another. There is an easy way to do that by squashing the array of arrays into one longer array with `flatten`:

In [ ]:
axes.flatten()

We can avoid copying the code for each year by looping over it. And then using string formatting with format strings to include the number we want. This is not the nicest way to do this for sure, but we already know how to do it with datetime strings and laziness often wins in the end.

In [ ]:
fig, axes = plt.subplots(2, 2, sharey=True, figsize=(12,12))
df = data_10min.rolling(5, center=True, win_type='triang').mean()
for ax, year in zip(axes.flatten(), range(17,21)):
    df.loc[f'20{year}-12-29':f'20{year+1}-01-03'].plot(ax=ax, xlabel='')
ax.set_ylim(0, 200) ;

<a id="new-year-with-line"></a>
###### New Year's Eve close-up
The last and first day of the year. To not miss in miss the year changing we add a line at midnight:

In [ ]:
fig, axes = plt.subplots(2, 2, sharey=True, figsize=(12,12))
df = data_10min.rolling(3, center=True, win_type='triang').mean()
for ax, year in zip(axes.flatten(), range(17,21)):
    df.loc[f'20{year}-12-31':f'20{year+1}-01-01'].plot(ax=ax, xlabel='')#rot=20)
    ax.axvline(f'20{year+1}-01-01 00:00', color='green', alpha=0.5)
ax.set_ylim(0, 200)

Or even closer, looking at just a 24h window:

In [ ]:
fig, axes = plt.subplots(2, 2, sharey=True, figsize=(12,12))
df = data_10min.rolling(3, center=True, win_type='triang').mean()
for ax, year in zip(axes.flatten(), range(17,21)):
    df.loc[f'20{year}-12-31 18:00':f'20{year+1}-01-01 6:00'].plot(
        ax=ax, marker='.', xlabel='')
    ax.axvline(f'20{year+1}-01-01 00:00', color='green', alpha=0.5)
ax.set_ylim(0, 200) ;

#### Within a day / by time of day

In [ ]:
data_10min['time'] = data_10min.index.time

In [ ]:
by_time = data_10min.groupby('time')
by_time.ngroups

In [ ]:
by_time.mean().plot()

The labels are at weird random times. Let's fix this manually:

In [ ]:
ax = by_time.mean().plot()
ax.set_xticks(range(0, 24*60*60+1, 3*60*60)) ; # time is treated like seconds

In [ ]:
time_stats = by_time.describe()
time_stats.T

<a id="hierarchical-column-index-part1"></a>
##### Interlude: Selecting columns in hierarchical indices (part 1)

When we just print the `time_stats` (without flipping it around using `.T`), we can see that we have two column name levels. This is called a hierarchical index (or Multilevel Index):

In [ ]:
time_stats.head()

If we try to plot the data like we usually do, we get a long scary looking error, telling us the column name 'mean' can't be accessed:

In [ ]:
time_stats.reset_index().plot('time', 'mean')

If we look at the columns, we can see that it has two levels:

In [ ]:
time_stats.columns

One way to fix the problem is to just specify both column levels with a tuple:

In [ ]:
time_stats.reset_index().plot('time', ('PM2.5','mean'))

But generally, it is much easier to first select one entry in the upper level and then we can work with it like we are used to:

In [ ]:
ax = time_stats['PM2.5'].plot(y=['mean', '50%'])
ax.set_xlim(time_stats.index[0], time_stats.index[-1])
ax.set_xticks(range(0, 24*60*60+1, 3*60*60))
ax.fill_between(time_stats.index, '25%', '75%', data=time_stats['PM2.5'],
                color='gray',alpha=0.3) ;

We'll see more ways to deal with a hierarchical column index [a bit later](#hierarchical-column-index-part2).

#### Within a week / by weekday and time of day

In [ ]:
%%time
weekday_stats = data_10min.groupby(data_10min.index.weekday).describe()
weekday_stats

But what do the day numbers mean? According to Pandas documentation the first day of the week by default is Monday and therefore has the value 0, Tuesday 1,...

From the calendar module we can get the short versions of the weekday (even localized if we want that). They also start the week with Monday.

In [ ]:
%%time
weekday_stats.index = calendar.day_abbr
weekday_stats.T # flip the table around to make it easier to read

The PM2.5 daily mean for Monday is by far the lowest. Let's double check we didn't mix up Monday with Sunday:

In [ ]:
today = pd.to_datetime('2021-06-17')
today.day_name(), today.weekday()

In [ ]:
last_monday = pd.to_datetime('2021-06-14')
last_monday.day_name(), last_monday.weekday()

In [ ]:
data_10min.groupby(data_10min.index.day_name())['PM2.5'].mean().sort_values()

Seems to be really the case that Monday is lowest for PM2.5. Maybe we can make sense of it by looking at how the pollution changes over the day for each weekday.

<a id="hierarchical-column-index1"></a>
##### Interlude: Grouping by more than one column
We can group by more than one value in Pandas. This will give us a hierarchical index (also called Multi Index) for the rows. As we have seen with already in the columns, hierarchical indices can be a bit of a pain to deal with.

In [ ]:
%%time
# this actually takes a few seconds
by_day_time = data_10min.groupby(
    [data_10min.index.weekday, data_10min.index.time])
day_time_stats = by_day_time.describe()
day_time_stats

In [ ]:
ax = day_time_stats['PM2.5'].plot(y=['mean', '50%'])

Let's try and figure out how the x-axis values work in the plot with this strange index, by playing around a bit.

In [ ]:
ax = day_time_stats['PM2.5'].plot(y=['mean', '50%'])
ax.axvline(0)
ax.axvline(24)
ax.axvline(24*6, color='gray')

Looks like the x-value is just the row number. We have 6 entries per hour, so one day is 24*6. Let's make the plot a bit easier to read.

In [ ]:
for pollutant in ['PM2.5', 'PM10']:
    ax = day_time_stats[pollutant].rolling(5, center=True).mean().plot(
        y=['mean', '50%'], title=pollutant)
    ax.set_xlim(0, 7*24*6)
    for i in range(1, 8):
        ax.axvline(i*24*6, color='gray')
    ax.axhline(data[pollutant].mean(), alpha=0.5)
    ax.axhline(data[pollutant].median(), color='C1', alpha=0.5)
    ax.set_xticks(range(12*6, 7*24*6, 24*6))
    ax.set_xticklabels(calendar.day_abbr)
    ax.set_xlabel('time of day and week day')
    ax.set_ylabel(pollutant) ;

In [ ]:
weekday_stats.loc[:, (['PM2.5', 'PM10'], ['mean', '50%'])].T

<a id="hierarchical-column-index-part2"></a>
##### Interlude: Selecting columns in hierarchical indices (part 2)

Accessing some value from the lower column index, but all from the upper, is a bit tricky.

There are two not too verbose alternatives to do that. Usually we select all entries by using `:`, but in a tuple like `(upper_columns,lower_columns)` we are not allowed to do `(:,lower_column)` as this is an invalid syntax in Python.

However, the `:` is just a short form for accessing slices in Python and we can use the more explicit form of `slice(None)` to select all entries.

In [ ]:
weekday_stats.loc[:, (['PM2.5', 'PM10'], ['mean', '50%'])]

In [ ]:
weekday_stats.loc[:, (slice(None), ['mean', '50%'])]

Or we just **swap the order of the columns** (temporarily or permanently).

In [ ]:
weekday_stats.swaplevel(axis='columns')

In [ ]:
weekday_stats.swaplevel(axis='columns')[['mean', '50%']]

##### Interlude: Changing a hierarchical row index into a normal index
We can also get rid of one layer in a hierarchical index, by removing it from the index. For the row index we can achieve this by calling `reset_index()` with the number or name of the level we want to remove.

To plot all the days in one plot with pandas, this form is more useful:

In [ ]:
day_time_stats['PM2.5'].head(3)

In [ ]:
day_time_stats['PM2.5'].reset_index(level=0).rename(
    columns={'timestamp': 'weekday'}).head(3)

In [ ]:
df = day_time_stats['PM2.5'].reset_index(level=0).rename(
    columns={'timestamp': 'weekday'})
df.groupby('weekday', sort=False)['mean'].plot(legend=True) ;

*We could actually do the groupby also with an index level (but we haven't done that yet).*

Let's try to make this plot a bit easier to read, by smoothing the data a bit and applying some styling to the plot.

We know we are only interested in the PM2.5 mean for now, so let's start with that:

In [ ]:
day_time_stats['PM2.5']['mean']

In this case we have two row index level, but no columns at all. It will be easier to work with if we move one level to the columns. We can do this with `unstack` (and we can also move a level of columns to the row index with `stack`).

In [ ]:
day_time_stats['PM2.5']['mean'].unstack(level=0)

It is better if we do the smoothing before the unstacking, then we can smooth the whole week instead of each day separately. This way we get less null values because we have only one beginning and end:

In [ ]:
pm25_by_weekday = day_time_stats['PM2.5']['mean'].rolling(5, center=True).mean()
pm25_by_weekday = pm25_by_weekday.unstack(level=0)
pm25_by_weekday.columns = calendar.day_abbr
pm25_by_weekday

##### Note: even more reshaping with pivot and melt
If you didn't get enough of moving index levels around or the methods above don't click for you there is two more common reshaping methods: `pivot` for going from long to wide (creating more columns and less rows) and `melt` for going from wide to long.

Some packages, especially ones related to statistics, like [seaborn](https://seaborn.pydata.org/) (a very handy plotting library for more statistical plots) expect "tidy data". This generally means a long format (many rows), rather than a wide one. Quick plotting with Pandas or Matplotlib on the other hand is easier with a wide format (a column per line or plot you want to draw). If you start using these other packages you probably need to get a bit familiar with the reshaping operations at some point. It can be quite tricky to wrap your head around it, especially if you use it only occasionally. In case it comforts you, doing the same thing in R was not any easier (at least the last time I used it).

I won't go into detail of how these reshaping methods work, as it would be a bit much even for this wild ride. Also, there are a lot of resources out there that explain it better with nice diagrams and all, like the [official Pandas reshaping tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html#long-to-wide-table-format) which, funnily enough, also uses air quality data. In that tutorials the `pivot_table` method is also described, which some of you might be familiar with from Excel or so. It is similar to `pivot`, but instead of just reshaping it also aggregates some of the data points.

<a id="plot-styling"></a>
##### Interlude: Plot styling

In [ ]:
pm25_by_weekday.plot()
plt.xticks(pm25_by_weekday.index[::6*3]) ;

Still difficult to read. Let's use different line styles with the `style` parameter. Let's use two different line styles for the weekdays and one for the weekend. In Python we can do some handy operations on list to repeat them or concatenate them:

In [ ]:
['a', 'b'] * 3 + ['x', 'y']

Let's also choose different plot colors. Matplotlib comes with quite a few color palettes, which are [listed and visualized](https://matplotlib.org/stable/gallery/color/colormap_reference.html) in the [Gallery](https://matplotlib.org/stable/gallery/) as one of the examples. In this case we have some order (Mon-Sun), but also a bit of discontinuity between weekdays and weekend. Let's try to find a color map that reflects that.

In [ ]:
pm25_by_weekday.plot(style=(['--',  ':']*3)[:5] + ['-']*2, lw=2.5,
                     colormap='summer')
plt.xticks(range(0, 24*60*60+1, 3*60*60)) ;
plt.xlim(0, 24*60*60)

Okayish, but Sunday is hard to read. To improve this, we can use a different matplotlib style for this plot, like one with a dark background, but let's first quickly put the plotting code into a function, so we don't have to copy it all the time:

In [ ]:
def plot_weekdays_together(ax=None):
    pm25_by_weekday.plot(style=(['--',  ':']*3)[:5] + ['-']*2, lw=2.5,
                         colormap='summer', ax=ax)
    plt.xticks(range(0, 24*60*60+1, 3*60*60))
    plt.xlim(0, 24*60*60)

In [ ]:
plt.style.available

In [ ]:
with plt.style.context('seaborn-darkgrid'):
    plot_weekdays_together()

Or if you don't like not having the box around the dark area (and can't be bothered to add it back, like me), maybe this one:

In [ ]:
with plt.style.context(['bmh']):
    plot_weekdays_together()

Or maybe you like it really dark:

In [ ]:
with plt.style.context('dark_background'):
    plot_weekdays_together()

## Getting the results out

### Sharing the exploration
If we want to share all our results with other people, we can just share the Jupyter Notebook itself. In case the audience might not have Python and Jupyter installed (or it would be too much effort to open it that way), you can get the whole notebook as **HTML file** by going to "File" -> "Export Notebook As..." -> "HTML". There are also quite a few possibilities to select what should get included, but many of them are currenlty only accessible from the command line. For example, you can hide all code in the html output with `jupyter nbconvert notebook_name.ipynb --no-input --to html`.

There are a lot of other formats there, but HTML is generally the easiest to share, apart from maybe PDF. However, **PDF files** requires LaTeX to be installed (and "Webpdf" at least needs additional packages installed). It might require some formatting work as well in the original notebook to ensure e.g. code lines are not too long and get cut off.

In case you want to use the code from your notebook in a script or in a larger program, you can export all the code in the notebook with "File" -> "Export Notebook As..." -> "Executable Script". This will give you a **Python file** with all the code, but it also includes the cell numbers and the markdown text cells as comments.

### Exporting plots
To save a plot to a file you can use `plt.savefig` to save the current figure in a large variety of formats:

In [ ]:
plot_weekdays_together()
plt.savefig('pm_by_weekday.pdf')

If you don't add any further arguments, there probably a lot of empty space around the plot. You can change that with setting the `bbox_inches` parameter for example to `'tight'`.

If only want to save the plot but not display it in the notebook, you can close the figure in the same cell (but after `savefig`). If you don't have a handle to the figure object, you can get it with `plt.gcf()` (get current figure).

In [ ]:
with plt.style.context('dark_background'):
    plot_weekdays_together()
plt.savefig('pm_by_weekday.pdf', bbox_inches='tight')
plt.close(plt.gcf())

If you save it as a png or other raster file format, you probably want to increase the `dpi` (dots per inch) a bit, especially if it is for printing:

In [ ]:
with plt.style.context('dark_background'):
    plot_weekdays_together()
plt.savefig('pm_by_weekday.png', dpi=300, bbox_inches='tight')

But if you just quickly want to share the png image with someone else or copy it somewhere, the quality of the plot in the notebook is often enough. You can quickly get that image with `Shift + Right-Click` on the image in the notebook and then choosing something like "Copy Image" from your browser menu. Usually you can copy images in the browser by just right-clicking on an image (without pressing `Shift`), but Jupyter Lab uses that for offering you other options, but at least it tells you what to do to get to the browser menu at the very bottom of that options list.

### Exporting Pandas dataframes
Pandas supports a lot of output formats. There are all the formats you would expect, like CSV, some you might hope for, like Excel, and some that you probably wouldn't expect, like to copy it to clipboard in a format that allows you to paste it into Excel:

In [ ]:
interesting_stats = year_stats['PM2.5'].drop(['count', 'std'], axis='columns')

In [ ]:
interesting_stats.to_clipboard(excel=True)

In [ ]:
interesting_stats.to_clipboard(excel=False, index_names=False)

Or for web developers:

In [ ]:
interesting_stats.to_json()

We can also get the dataframe tables as HTML. The `to_html` output is quite long (as it is nicely formatted), so you might want to collapse the output by clicking on the long blue bar next to the output that you get when you click on the cell.

In [ ]:
print(interesting_stats.to_html(index_names=False))

We can copy the HTML output into Jupyter markdown cells as well. Could be quite handy to referring to some data from another notebook, right? This is what you get:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>count</th>
      <th>mean</th>
      <th>std</th>
      <th>min</th>
      <th>25%</th>
      <th>50%</th>
      <th>75%</th>
      <th>max</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2017</th>
      <td>96750.0</td>
      <td>6.021118</td>
      <td>5.011754</td>
      <td>0.3</td>
      <td>2.40</td>
      <td>4.60</td>
      <td>8.27</td>
      <td>158.83</td>
    </tr>
    <tr>
      <th>2018</th>
      <td>174778.0</td>
      <td>8.989901</td>
      <td>7.295606</td>
      <td>0.3</td>
      <td>3.70</td>
      <td>7.10</td>
      <td>12.60</td>
      <td>194.97</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>164439.0</td>
      <td>8.224983</td>
      <td>7.543542</td>
      <td>0.3</td>
      <td>2.93</td>
      <td>6.23</td>
      <td>11.27</td>
      <td>253.73</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>199255.0</td>
      <td>7.126150</td>
      <td>6.378018</td>
      <td>0.3</td>
      <td>2.43</td>
      <td>4.93</td>
      <td>10.40</td>
      <td>246.83</td>
    </tr>
    <tr>
      <th>2021</th>
      <td>89974.0</td>
      <td>6.808873</td>
      <td>6.330431</td>
      <td>0.3</td>
      <td>2.30</td>
      <td>5.23</td>
      <td>9.83</td>
      <td>280.87</td>
    </tr>
  </tbody>
</table>

Like with all the outputs (apart from the clipboard) we can save it directly to a file. We can also open that file in Jupyter Lab. You'll notice that the HTML file doesn't look that nice outside of a Jupyter Notebook. This is because the styling information (CSS markup) is not part of the output.

In [ ]:
interesting_stats.to_html('interesting_stats.html', index_names=False)

#### One last goody
In case you are a big $\LaTeX$ lover or even just an occasional user, here is one last goody for you:

In [ ]:
print(interesting_stats.to_latex())

Instead of printing you probably want to save it as a tex file though and then include it in your $\LaTeX$ file with `\input{name_of_exported.tex}`:

In [ ]:
month_table.T.to_latex('interesting_stats.tex')

# Closing words
## Notebook best practice advice

Often when you finish working on a notebook it makes sense to restart the session and executing everything again (automatically or manually) or alternatively right clicking on the notebook name in the file browser in the left toolbar and selecting "Duplicate".

Starting execution from scratch helps with catching situations like variable names that were changed over time, but the old one was still available and so you didn't notice it or that you accidentally changed the execution order of the cells.

In case you want to share your notebook with other people or want to turn it into nicer Python code to use in other places, you probably need to go over your notebook and clean it up a bit. For example, if you copied a cell several times and then just made small changes to the original cell, you probably want to extract the code lines and turn it into a proper function. Depending on how big and messy your original notebook is, you could either duplicate it and then do the changes in that copy or start a new notebook and copy the relevant parts over, e.g. by drag and dropping the relevant cells to the new notebook (while both are open in a split view).

## Moving on from here
If you made it all the way down here congratulations! I appreciate you taking the time to look and work through something new.

But now that you are done, what's next?

### Goodbye Python?
Maybe you'll forget all about Python and Pandas, that's okay. Maybe a time will come up when you happen upon a problem that could be solved well with Python and you remember it then. Or whatever you learned here might help you in another programming language. If you end up doing some data analysis with a different language (like R or Julia), that will probably be the case.

### Hello Notebook?
Even if you forget all about anything snake related and never need anything of this again, I hope that the concept of "computational notebooks" like this Jupyter Notebook will inspire you a bit. Don't forget that they can also be used with other languages, even some that might surprise you at first (like C++ or C#).

I think this way of combining code with data, explanations and visualizations is a great way for exploring something new and also for archiving it. Even if it is some small throw-away exploration and you think you'll never come back to it, you might get surprised. It's quite likely that you'll come back to some earlier parts during the exploration and even afterwards it at least happened to me a few times already that I came back to look at the code or one of the plots because I needed it again.

In my opinion it is also a great for learning and teaching. I'm convinced that without Jupyter Notebooks some of the plentiful and high-quality learning resources and package documentation in the Python world would not exist. And for the learner the ability to just play around a bit with the code, tweak a number here, try something different there, is opening up a whole new world.

Even if you are not using any material prepared by others, notebooks (and consoles) can be a great way to figure out how to do something in an area that you are not yet familiar with. With tab completion, access to the help strings and documentation, coupled with googling stuff and finding answers close to your problem, you can start to get into a kind of dialog with the programming language and the data. The notebook can be a recording of that dialog in case you want to retrace some of your steps or reflect on it. Or you can tidy it up a bit and help somebody else make a similar journey. You can also just extract what you learned either as code (by exporting as "Executable Script") or maybe just the insight, ideas or algorithms you discovered.

### Or travelling on?
In case you are still or (hopefully) more interested in Python and want to look at it a bit more closely, there are many paths you could take to continue your journey from here.

One could be to go through this notebook again, but trying it with a [different sensor](https://maps.sensor.community/), maybe one closer to you, and comparing it with this one, trying to change the code where needed.

Maybe you are sick and tired of PM data (it *was* a lot). In that case there are so many resources out there, many of them freely available. Quite a few books are available both as texts to read, as well as Jupyter notebooks to go through (in case you enjoyed it with this one and even otherwise: most of them take you along a much gentler journey).

There is also a ton of talks and [tutorial recordings](https://pyvideo.org/tag/tutorial/) available from hundreds of Python conferences. If you like learning from videos head over to [PyVideo.org](https://pyvideo.org/) where you'll find over 10 000 Videos in English and nearly 200 in German. There is also a real flood of online guides and tutorials. One that stands out a bit in quality and in scope is "Real Python", like this [introduction to Pandas dataframes](). They do also have a [list of learning paths](https://realpython.com/learning-paths/) that might inspire or help guide you on your way forward (though be warned, most of their video series have only the first few videos freely available, but all written guides I have seen there were free).

At least in the data science space the official tutorials of packages have increased a lot in quality in the last few years. So, don't miss out on looking at their tutorials, even if you are just a casual user. I sure wish I had looked at the start of the [basic concepts of Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html) earlier, as there they describe quite concisely what each part in a figure is called or the two different main styles of using matplotlib. However, these package tutorials tend to assume more familiarity with Python (and often even the package itself) than most books and other online guides, so they are probably not the best starting point.